In [1]:
cd ..

/model-data


In [2]:
from pipeline.segmentation import *
from eighth_try.mobilenetv2 import *

import collections
import hashlib
import os.path
import random
import re
import sys

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import keras
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import Model
from keras.layers import Dense, Input, LSTM
from pathlib import Path
import sklearn
from sklearn import datasets
from sklearn import metrics
import shutil
from keras import initializers
from keras.models import load_model
import pickle
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.image import load_img, array_to_img, img_to_array
from IPython.display import Image
from IPython.display import display
import cv2
import matplotlib.pyplot as plt
#from PIL import Image

Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.


W0711 13:50:01.669624 140274189141760 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [3]:
def get_pretrained_base_model_for_bottlenecks(show_model_summary = False):
    model = MobileNetV2()
    if show_model_summary:
        model.summary()
    base_model = Model(inputs=model.input, outputs=model.get_layer('global_average_pooling2d_1').output)
    print("Base_model_loaded 100 percent")
    return base_model

In [4]:
action_model=load_model("eighth_try/human_seg_rnn_mask_model.h5")
mobilenet_model=get_pretrained_base_model_for_bottlenecks(False)
print("Action Model Loaded")
class_names = ['ABOVE', 'ACCIDENT', 'ADVISE_or_INFLUENCE', 'AFTERNOON', 'AGAIN', 'ALLIGATOR', 'ANSWER' ,'A_LOT', 'BECAME', 'COME_HERE', 'EXCEED', 'GENERATION', 'LESS_THAN', 'OF' ,'THREAT', 'TOTAL']


Instructions for updating:
Colocations handled automatically by placer.


W0711 13:50:04.315125 140274189141760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0711 13:50:05.212991 140274189141760 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


W0711 13:50:06.967204 140274189141760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Base_model_loaded 100 percent
Action Model Loaded


In [5]:
video_path = "testing_videos"
frame_target_dir = "testing_frames"
gif_dir = "testing_gif"


In [6]:
def video_to_frames_other(video_path,frame_target_dir):
    #print(video_path)
    vidObj = cv2.VideoCapture(video_path)
    count = 0
    success = 1
    ensure_folder_exists(frame_target_dir)
    names=[]
    while success: 
        success, image = vidObj.read() 
        if image is None:
            continue
        framename=frame_target_dir+'/' + str(count)+".jpeg"
        cv2.imwrite(framename,image)
        names.append(framename)
        count += 1
    #print("Video to frame done")
    print("Number of frames = " ,count)
    return names

In [7]:
def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [8]:
def frames_to_segmented_frames(names):
    frames=[]
    for name in names:
        img=load_img(name)
        img=img_to_array(img)
        frames.append(img)
    #print("Segmentation done")
    return human_segmentation(np.asarray(frames))

In [9]:
def action_prediction(action_frames):
    action_frames=np.asarray(action_frames)
    action_frames=action_frames[:,:,:,:]
    action_frames = preprocess_input(action_frames)
    new_frames=[]
    # resize command
    for i in range(action_frames.shape[0]):
        new_frames.append(cv2.resize(action_frames[i,:,:,:], dsize=(224, 224), interpolation=cv2.INTER_CUBIC))
    action_frames=np.asarray(new_frames)    
    features=mobilenet_model.predict(action_frames)
    features=np.reshape(features,(1,features.shape[0],features.shape[1]))
    prediction=action_model.predict(features)
    #print(prediction)
    class_predicted=class_names[prediction.argmax()]
    print(class_predicted)

In [10]:
ensure_folder_exists(frame_target_dir)

In [16]:
for video in os.listdir(video_path):
    video_path_cat = os.path.join(video_path,video)
    frame_target_dir_cat = os.path.join(frame_target_dir,video[:-4])
    if(video_path_cat.endswith(".mp4")):
        display(Image(url='testing_gif/' + video[:-4] + '.gif'))
        names=video_to_frames_other(video_path_cat,frame_target_dir_cat)
        frames=frames_to_segmented_frames(names)
        frames=np.asarray(frames)
        action_prediction(frames)

Number of frames =  31
A_LOT
